In [8]:
import numpy as np

def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.array([[1,2],
              [3,4],
              [5,6]]) #3x2

b = np.array([[7,8,9],
              [10,11,12]]) #2x3



assert np.array_equal(a@b, my_matmult(a,b))
assert np.array_equal(b@a, my_matmult(b,a))


In [9]:
def tenmult(m1, m2):
    pass


A = np.array([
  [[1,2,3],    [4,5,6],    [7,8,9]],
  [[11,12,13], [14,15,16], [17,18,19]],
  [[21,22,23], [24,25,26], [27,28,29]],
  ]) # [3,3,3]
B = np.array([
  [[1,2,3],    [4,5,6],    [7,8,9]],
  [[11,12,13], [14,15,16], [17,18,19]],
  [[21,22,23], [24,25,26], [27,28,29]],
  ]) # [3,3,3]

# C = A @ B # [3,3,3]
# C = np.tensordot(A, B) # [3,3]
C = np.matmul(A,B) # [3,3]
print(C.shape)
print(C)

(3, 3, 3)
[[[  30   36   42]
  [  66   81   96]
  [ 102  126  150]]

 [[ 510  546  582]
  [ 636  681  726]
  [ 762  816  870]]

 [[1590 1656 1722]
  [1806 1881 1956]
  [2022 2106 2190]]]


In [10]:
import tensorflow as tf

A = tf.constant([
  [[1,2,3],    [4,5,6],    [7,8,9]],
  [[11,12,13], [14,15,16], [17,18,19]],
  [[21,22,23], [24,25,26], [27,28,29]],
  ]) # [3,3,3]
B = tf.constant([
  [[1,2,3],    [4,5,6],    [7,8,9]],
  [[11,12,13], [14,15,16], [17,18,19]],
  [[21,22,23], [24,25,26], [27,28,29]],
  ]) # [3,3,3]

print(A)
print(B)

print(tf.matmul(A,B)) # [3,3]

tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 12 13]
  [14 15 16]
  [17 18 19]]

 [[21 22 23]
  [24 25 26]
  [27 28 29]]], shape=(3, 3, 3), dtype=int32)
tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 12 13]
  [14 15 16]
  [17 18 19]]

 [[21 22 23]
  [24 25 26]
  [27 28 29]]], shape=(3, 3, 3), dtype=int32)
tf.Tensor(
[[[  30   36   42]
  [  66   81   96]
  [ 102  126  150]]

 [[ 510  546  582]
  [ 636  681  726]
  [ 762  816  870]]

 [[1590 1656 1722]
  [1806 1881 1956]
  [2022 2106 2190]]], shape=(3, 3, 3), dtype=int32)
